# Triangular distribution

The [Triangular](https://en.wikipedia.org/wiki/Triangular_distribution) distribution is specified via an interval $[a,b]$ and a mode $c \in [a, b]$, though I will be modelling $a < c < b$.



In [ ]:
from triangular import Triangular
import torch
import numpy as np
import math
from matplotlib import pyplot as plt
from triangular import mc_kl_triangular_triangular, kl_triangular_triangular

In [ ]:
def plot_pdf(modes, step=1e-3):
    plt.title('Triangular(0, 1).pdf')
    x = np.arange(0. + step, 1., step)
    for mode in modes:
        p_x = Triangular(mode=torch.ones(1) * mode)
        y = np.exp(p_x.log_prob(torch.from_numpy(x).float()).numpy())
        plt.plot(x, y)
    plt.legend(['c=%.2f' % c for c in modes], 
               loc='upper center', bbox_to_anchor=(0.5, -0.1),
               fancybox=True, shadow=True, ncol=4)
    
def plot_cdf(modes, step=1e-3):
    plt.title('Triangular(0, 1).cdf')
    x = np.arange(0. + step, 1., step)
    for mode in modes:
        p_x = Triangular(mode=torch.ones(1) * mode)
        y = p_x.cdf(torch.from_numpy(x).float()).numpy()
        plt.plot(x, y) 
    plt.legend(['c=%.2f' % c for c in modes], 
               loc='upper center', bbox_to_anchor=(0.5, -0.1),
               fancybox=True, shadow=True, ncol=4)
    
def plot_icdf(modes, step=1e-3):
    plt.title('Triangular(0, 1).cdf')
    u = np.arange(0. + step, 1., step)
    for mode in modes:
        p_x = Triangular(mode=torch.ones(1) * mode)
        x = p_x.icdf(torch.from_numpy(u).float()).numpy()
        plt.plot(u, x)
    plt.legend(['c=%.2f' % c for c in modes], 
               loc='upper center', bbox_to_anchor=(0.5, -0.1),
               fancybox=True, shadow=True, ncol=4)
    
def plot_samples(modes, n=1000, cols=3):
    plt.subplots_adjust(wspace=0.5, hspace=0.6)
    plt.suptitle('Triangular(0, 1).cdf')
    for i, mode in enumerate(modes, 1):        
        p_x = Triangular(mode=torch.ones(1) * mode)
        x = p_x.rsample(sample_shape=torch.Size([n])).numpy()        
        plt.subplot(math.ceil(len(modes)/cols), cols, i)
        plt.title('c=%.2f' % mode)
        _ = plt.hist(x, bins=50, density=True)        
        
def plot_entropy(modes):
    y1 = []
    y2 = []
    for mode in modes:
        p_x = Triangular(torch.ones(1) * mode)
        y1.append(p_x.entropy().numpy())
        y2.append(p_x.cross_entropy(p_x).numpy())
    y1 = np.array(y1).flatten()
    y2 = np.array(y2).flatten()
    plt.title('Entropy')
    plt.plot(modes, np.around(y1, 4), 'o', fillstyle='none')
    plt.plot(modes, np.around(y2, 4), '+', fillstyle='none')
    plt.legend(['wikipedia', 'wilker'])
    #for a, b, c in zip(modes, y1, y2):
    #    print('%.2f %.4f %.4f' % (a, b, c))
    
def plot_kl(modes, ref=1., upper=1., n_samples=100, repetitions=100):
    
    p_ref = Triangular(mode=torch.ones(1) * ref)
    x = []
    y1 = []
    y1_exact = []
    y2 = []
    y2_exact = []
    plt.title('KL divergence')
    for mode in modes:
        p_x = Triangular(torch.ones(1) * mode)
        x.append(mode)
        kl1 = [mc_kl_triangular_triangular(p_ref, p_x, n_samples=n_samples).numpy() for _ in range(repetitions)]
        y1.append(kl1)
        kl2 = [mc_kl_triangular_triangular(p_x, p_ref, n_samples=n_samples).numpy() for _ in range(repetitions)]
        y2.append(kl2)
        kl1_exact = kl_triangular_triangular(p_ref, p_x).numpy()
        y1_exact.append(kl1_exact)
        kl2_exact = kl_triangular_triangular(p_x, p_ref).numpy()        
        y2_exact.append(kl2_exact)
    y1 = np.array(y1).reshape([len(modes), -1])
    y2 = np.array(y2).reshape([len(modes), -1])
    y1_exact = np.array(y1_exact).reshape([len(modes)])
    y2_exact = np.array(y2_exact).reshape([len(modes)])
    
    # MC
    plt.boxplot([y1[i] for i, c in enumerate(modes)], 
                positions=np.arange(1, len(modes) + 1) - 0.2,
                widths=0.2,  patch_artist=True,
                boxprops=dict(facecolor='blue', color='blue'),
                zorder=-1)
    # exact
    plt.plot(np.arange(1, len(modes) + 1) - 0.2, y1_exact, '*', color='red')
    
    # MC
    plt.boxplot([y2[i] for i, c in enumerate(modes)], 
                positions=np.arange(1, len(modes) + 1) + 0.2,
                widths=0.2, patch_artist=True,
                boxprops=dict(facecolor='orange', color='orange'),
                zorder=-1)
    # exact
    plt.plot(np.arange(1, len(modes) + 1) + 0.2, y2_exact, '*', color='red')
    
    plt.xticks(np.arange(1, len(modes) + 1), ['%.2f' % c for c in modes])
    plt.ylabel('KL')
    plt.xlabel('c')
    plt.legend(['KL-MC(Tri(%.2f)||Tri(c))' % ref,
                'KL(Tri(%.2f)||Tri(c))' % ref,
                'KL(Tri(c)||Tri(%.2f))' % ref],
               loc='upper center', bbox_to_anchor=(0.5, -0.1),
               fancybox=True, shadow=True, ncol=4)

The entropy is not a function of the mode

In [ ]:
plot_entropy(np.arange(0.1, 1., 0.15))

In [ ]:
plot_pdf(np.arange(0., 1.1, 0.1))

In [ ]:
plot_cdf(np.arange(0., 1.1, 0.1))

In [ ]:
plot_icdf(np.arange(0., 1.1, 0.1))

In [ ]:
plot_samples(np.arange(0.1, 1., 0.15))

In [ ]:
plot_kl(np.arange(0., 1.1, 0.1), ref=0.5, n_samples=1, repetitions=100)

In [ ]:
plot_kl(np.arange(0., 1.1, 0.1), ref=0.5, n_samples=100, repetitions=100)